# Line Following using Sensors
**Full Coding available at [object_tracking.py](object_tracking.py)** 
<br>
This is the documentation for Object Trcking using OpenCV (Color Base) 

***Libraries Used In The Script***
- Time 
- Motor_Encoder
- PCA9685_MC 
- OpenCV 
- Picamera2 
- Libcamera 
- Numpy



## Installing Libraries @ Dependencies 
- Build-In Libraries 
    - Picamera2 
    - libcamera
    - time 
    - PCA9685_MC 
    - Motor Encoder 

- User Installed Libraries
    - OpenCV 
    - numpy 

### First, let's install the user install libraries ! 
1. OpenCV 
    - Follow the link [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
2. NumPy 
    - Follow the link [numpy.org](https://numpy.org/install/)


### Verify the installed libraries 
1. OpenCV 
    - In Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed
      
2. NumPy 
    - In Python File type: <br>
    `import numpy as np`
    - Installation sucess if coding run without error 

## Let's Start Coding ! 
### 1. Import all the libraries 
- OpenCV 
- NumPy 
- Picamera2 
- libcamera
- Motor_Encoder 
- PCA9685_MC
- time

In [ ]:
import cv2
import time
from picamera2 import Picamera2
from libcamera import controls
from PCA9685_MC import Motor_Controller
from Motor_Encoder import Encoder
import numpy as np

### 2. Initialise all the libraries in a function 
- PCA9685_MC (For Motor Driver and Servo HAT)
- Motor_Encoder (For Back Motoe Encoder)
- Picamera2 (To access the camera)
- libcamera (To control the camera)

In [ ]:
picam = Picamera2()
picam.configure(picam.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
picam.start()
picam.set_controls({"AfMode": controls.AfModeEnum.Continuous})
Motor = Motor_Controller()
enc = Encoder()

### 3. The `colorPicker` function 
- This function enables user to pick the traget color to be tracked 
- User will use the trackbar to adjust the HSV value 
- This function will create a window and  preview for the user to adjust the HSV color value 
- This function will return the **Lower Bounderies** and **Upper Bounderies** of HSV color Range. 


In [ ]:
def colorPicker(): 
    global picam 
    def nothing(x):
        pass    
    cv2.namedWindow("Color_Picker")
    cv2.createTrackbar("Lower Hue", "Color_Picker", 0, 179, nothing)
    cv2.createTrackbar("Lower Saturation", "Color_Picker", 0, 255, nothing)
    cv2.createTrackbar("Lower Value", "Color_Picker", 0, 255, nothing)
    cv2.createTrackbar("Upper Hue", "Color_Picker", 179, 179, nothing)
    cv2.createTrackbar("Upper Saturation", "Color_Picker", 255, 255, nothing)
    cv2.createTrackbar("Upper Value", "Color_Picker", 255, 255, nothing)
    while True:
        img = picam.capture_array()
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        l_h = cv2.getTrackbarPos("Lower Hue", "Color_Picker")
        l_s = cv2.getTrackbarPos("Lower Saturation", "Color_Picker")
        l_v = cv2.getTrackbarPos("Lower Value", "Color_Picker")
        u_h = cv2.getTrackbarPos("Upper Hue", "Color_Picker")
        u_s = cv2.getTrackbarPos("Upper Saturation", "Color_Picker")
        u_v = cv2.getTrackbarPos("Upper Value", "Color_Picker")
        lower_bound = np.array([l_h, l_s, l_v])
        upper_bound = np.array([u_h, u_s, u_v])
        mask = cv2.inRange(hsv, lower_bound, upper_bound) 
        res = cv2.bitwise_and(img, img, mask=mask)

        cv2.imshow("Color_Picker", mask)
        cv2.imshow("Color_Picker", img)
        cv2.imshow("Color_Picker", res)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
    return lower_bound, upper_bound 


### 4. The `main` function 
- This is the main function for the object tracking based on HSV color space. 
- Obtain the **Lower Bounderies** and **Upper Bounderies** HSV value from the `colorPicker()` function 
- In the ***while True*** loop 
    - Obtaine the image using picamera2 library 
        ```py
        img = picam.capture_array()
        ```
    - convert the image obtained to HSV color space 
        ```py
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        ```
    - Filter the hsv value from the image based on the lower_bounderies and the uper_bounderies 
        - This will filter out all the color except the target color 
        - The result of the image is black and white, where the black color is the filtered color and the white is the target color 
            ```py 
            mask = cv2.inRange(hsv, lower_bound, upper_bound)
            ``` 
    - Pass the image to OpenCV algrothim to recognise the target object and return the result as countours 
        ```py
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ```
    - If contours is deteceted 
        - calculate the area of the contours 
        - for all the contours that is more than 1500px 
        - Get the x and y coordinates 
        - Get the width and height of the contour 
        - Draw a bounding box on the detected tagret color 
        - Using the coordinates and the size of the contour (detected Object)
        - Calculate the center point of the object (X axis and Y axis )
        - Display the calculated coordinates and the area of the detected object  
        - Based on the center coordinates 
            - When the value of y axis is less than 400 the programe will start the object tracking program else the cr will not move. 
                - If the value of x axis is more than 50 but less than 300, turn the car clockwise (Right)
                - If the value of x axis is more than 400 but less than 600, turn the car AntiClockwise (Left)
                - if thr value of x axis is more than or equal to 320 but less than or equal to 400, the car will proceed forward 
                - If all the condition above is not met, the car will brake. 
    
    - Display the mask result and the final result in diffrent windows 
    - When the button q is pressed on openCV  window, the programe will end. 
    


In [ ]:
def main():
    global picam 
    lower_bound , upper_bound = colorPicker()

    while True:
        img = picam.capture_array()
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            for i, contour in enumerate(contours):
                area = cv2.contourArea(contour)
                if area > 1500:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(img, f"Object {i + 1}", (x, y - 10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)

                    center_x = int(x + w // 2)
                    center_y = int(y + h // 2)
                    print("Center X:", center_x)
                    print("Center Y:", center_y)
                    print("Area:", area)

                    cv2.putText(img, f"Center X: {center_x}", (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(img, f"Center Y: {center_y}", (10, 60), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(img, f"Area: {area}", (10, 90), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)

                    if center_y < 300:
                        if 50 < center_x < 320:
                            print("Turn right")
                            Motor.Clock_Rotate(20)
                        elif 400 < center_x < 600:
                            print("Turn left")
                            Motor.AntiClock_Rotate(20)
                        elif 320 <= center_x <= 400:
                            Motor.Forward(20)
                            print("Centered")
                        else:
                            print("Out of range")
                            Motor.Brake() 
                    else:
                        print("Out of range")
                        Motor.Brake()

        cv2.imshow("Camera", mask)
        cv2.imshow("Result", img)
        if cv2.waitKey(1) == ord('q'):
            break

### 4. Wraping up 
- when the programe start run the main function 
    ```py
    try:
        if __name__ == '__main__': 
            main()
    ``` 
- When there is keyboard interupt (Ctrl + c)
    ```py
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
    ```
- When the programe ends 
    ```py 
    finally:
    picam.stop()
    Motor.cleanup()
    enc.stop()
    cv2.destroyAllWindows()
    print("Program Terminated \n Exiting....")
    ``` 

In [ ]:
try:
    if __name__ == '__main__':
        main()
except KeyboardInterrupt:
    print("KeyboardInterrupt")
finally:
    picam.stop()
    Motor.cleanup()
    enc.stop()
    cv2.destroyAllWindows()
    print("Program Terminated \n Exiting....")
